#### Step 1: Load input texts and ground truth labels

In [ ]:
from parse_entities import *
import pandas as pd
import json
model = '[Your model here]'

In [ ]:
def make_batch(text_list:List[str], identifier_list:List[str],store_directory:str)-> None: 
    system_prompt =  "You are an expert in labeling Personally Identifiable Information. Start your response rightaway without adding any prefix(such as Response:) and suffix"
    ins_prompt = 'Label the entity of the following text: use @@@,### to label name\n'
    model= model
    try:
        os.makedirs(store_directory, exist_ok=True)
        print(f"Directory '{store_directory}' is ready.")
    except Exception as e:
        print(f"Error creating directory '{store_directory}': {e}")
    res = []
    for i in range(len(text_list)): 
        request = {"custom_id": f'{identifier_list[i]}', "method": "POST", 
                  "url": "/v1/chat/completions", 
                  "body": {"model": model, "messages": [{"role": "system", "content": system_prompt},{"role": "user", "content": ins_prompt + text_list[i]}],
                  "temperature":0}}
        res.append(request)
    
    store_directory += '/batch_request.jsonl'
    with open(store_directory, 'w',encoding='utf-8') as f:
        for entry in res:
            json_line = json.dumps(entry, ensure_ascii=False)
            f.write(json_line + '\n')

In [5]:
with open('new_texts.txt', 'r') as f:
    input_texts = json.load(f)

In [7]:
# Provide the directory to store the temporary files
gpt_storage_directory = "gpt_output_gpt4omini+ft"

identifier_list = [f'doc{i}' for i in range(len(input_texts))]

In [8]:
#make batch file for submitting finetune job 
make_batch(input_texts, identifier_list,gpt_storage_directory)

#submit finetune job 
submit_finetune_job(gpt_storage_directory)

#print the progress
check_progress(gpt_storage_directory)

Directory 'gpt_output_gpt4omini+ft' is ready.
Batch(id='batch_67444c25efac8190a3e9399cae8e3ca2', completion_window='24h', created_at=1732529190, endpoint='/v1/chat/completions', input_file_id='file-XMpFRyEmWvdiq3RAMkAH7E', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1732615590, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))


In [ ]:
#check progress, if completed, then retrieve the file_id
out_id = check_progress(gpt_storage_directory)
if out_id != None: 
    #if comleted, retrieve the result and store the result in a file.
    file_response = client.files.content(out_id)
    save_file_response_as_jsonl(file_response, gpt_storage_directory)  
    data_list = read_jsonl_file(gpt_storage_directory)
    data = resort_data(data_list,identifier_list)
    batch_extract_parse_store(data,identifier_list,gpt_storage_directory)

In [ ]:
detected_entities = extract_entities(r'gpt_output_gpt4omini+ft/gpt_result.json')
detected_entities

In [ ]:
df = pd.DataFrame(detected_entities, columns=['file_idx', 'entity_text', 'positions'])
df

In [15]:
output_file = 'results/TSCC_detected_gpt4omini+ft.csv'
df.to_csv(output_file, index=False)

print(f"CSV file saved as {output_file}")

CSV file saved as results/TSCC_detected_gpt4omini+ft.csv
